In [1]:
import sys
import os
import mido

In [2]:
from mido import MidiFile
import os
import sys
from torch.utils.data import Dataset, DataLoader

In [3]:
ourmidi = MidiFile('lakh/lmd_full/f/fffb8c7e68f92ce16c3e6755dde22f12.mid')

In [4]:
for i, track in enumerate(ourmidi.tracks):
    print('Track {}: {}'.format(i, track.name))
    for msg in track:
        print(msg.type)

Track 0: 
time_signature
sequencer_specific
set_tempo
end_of_track
Track 1: Strings 1
track_name
control_change
program_change
control_change
key_signature
control_change
note_on
note_on
note_off
note_on
note_off
note_on
note_off
note_on
note_off
note_on
note_off
note_on
note_off
note_on
note_off
note_on
note_off
note_on
note_off
note_on
note_off
note_on
note_off
note_on
note_off
note_on
note_off
note_on
note_off
note_on
note_off
note_on
note_off
note_on
note_off
note_on
note_off
note_on
note_off
note_on
note_off
note_on
note_off
note_on
note_off
note_off
note_on
note_on
note_off
note_on
note_off
note_on
note_off
note_on
note_off
note_on
note_off
note_on
note_off
note_on
note_off
note_on
note_off
note_on
note_off
note_on
note_off
note_on
note_off
note_on
note_off
note_on
note_off
note_on
note_off
note_on
note_off
note_on
note_off
note_on
note_off
note_on
note_off
note_on
note_off
note_on
note_off
note_on
note_off
note_on
note_off
note_on
note_off
note_off
control_change
note_on
note_of

In [5]:
import torch
import torch.nn.functional as functional

class MIDITrackError(Exception):
    pass

class MessageSequence:
    def __init__(self, mid):
        self.messages = []
        try: 
            for i in mid.tracks[1]:
                #print(i.type)
                if i.type in ['note_on', 'note_off']:
                    self.messages.append(i)
        except IndexError:
            raise MIDITrackError
        
        #calculate note durations
        timecounter = 0
        notedict = {}
        real_sequence = []
        for message in self.messages:
            timecounter += message.time
            if message.type == "note_on":
                notedict[message.note] = timecounter
                
            if message.type == "note_off":
                duration = timecounter - notedict[message.note]
                real_sequence.append((message.note, notedict[message.note], message.time, duration))
                
        self.sequence = real_sequence
        
    def midi_reencode(self):
        reencoded = []
        active_notes = {}
        timecounter = 0
        for (note, timestamp, offset, duration) in self.sequence:
            note_order = sorted(active_notes.keys(), key=lambda x: active_notes[x][0]) #timestamp is tuple item 0
            for active_note in note_order:
                if active_notes[active_note][0] < timestamp:
                    reencoded.append(mido.Message("note_off", 
                                                  channel=1, 
                                                  note=active_note, 
                                                  velocity=95, 
                                                  time=active_notes[active_note][1]))
                    timecounter += active_notes[active_note][1]
                    del active_notes[active_note]
            reencoded.append(mido.Message("note_on", 
                                          channel=1, 
                                          note=note,
                                          velocity=95, 
                                          time=timestamp-timecounter))
            active_notes[note] = (timestamp+duration, offset, duration)
            timecounter = timestamp

            
        note_order = sorted(active_notes.keys(), key=lambda x: active_notes[x][0]) #timestamp is tuple item 0
        for active_note in note_order:
            reencoded.append(mido.Message("note_off", 
                                          channel=1, 
                                          note=active_note, 
                                          velocity=95, 
                                          time=active_notes[active_note][1]))
            del active_notes[active_note]
            
        return reencoded
    
    def vector_encode(self):
        note_db = functional.one_hot(torch.arange(0, 128))
        encoded = []
        for (note, _, offset, duration) in self.sequence:
            note_vec = note_db[note].clone().detach()
            if offset > 100:
                offset = 100
            if duration > 4000:
                duration = 4000
            
            offset = offset/100
            duration = duration/4000
            
            encoded.append(torch.cat((note_vec, torch.Tensor([offset]), torch.Tensor([duration]))))
        return encoded

In [6]:
x = MessageSequence(ourmidi)

In [7]:
x.messages, x.sequence

([Message('note_on', channel=1, note=62, velocity=95, time=4608),
  Message('note_on', channel=1, note=64, velocity=95, time=96),
  Message('note_off', channel=1, note=62, velocity=95, time=54),
  Message('note_on', channel=1, note=65, velocity=95, time=138),
  Message('note_off', channel=1, note=64, velocity=95, time=32),
  Message('note_on', channel=1, note=64, velocity=95, time=1408),
  Message('note_off', channel=1, note=65, velocity=95, time=54),
  Message('note_on', channel=1, note=60, velocity=95, time=330),
  Message('note_off', channel=1, note=64, velocity=95, time=66),
  Message('note_on', channel=1, note=53, velocity=95, time=126),
  Message('note_off', channel=1, note=60, velocity=95, time=38),
  Message('note_on', channel=1, note=62, velocity=95, time=2266),
  Message('note_off', channel=1, note=53, velocity=95, time=40),
  Message('note_on', channel=1, note=64, velocity=95, time=56),
  Message('note_off', channel=1, note=62, velocity=95, time=24),
  Message('note_on', cha

In [8]:
x.midi_reencode()

[Message('note_on', channel=1, note=62, velocity=95, time=4608),
 Message('note_on', channel=1, note=64, velocity=95, time=96),
 Message('note_off', channel=1, note=62, velocity=95, time=54),
 Message('note_on', channel=1, note=65, velocity=95, time=138),
 Message('note_off', channel=1, note=64, velocity=95, time=32),
 Message('note_on', channel=1, note=64, velocity=95, time=1408),
 Message('note_off', channel=1, note=65, velocity=95, time=54),
 Message('note_on', channel=1, note=60, velocity=95, time=330),
 Message('note_off', channel=1, note=64, velocity=95, time=66),
 Message('note_on', channel=1, note=53, velocity=95, time=126),
 Message('note_off', channel=1, note=60, velocity=95, time=38),
 Message('note_on', channel=1, note=62, velocity=95, time=2266),
 Message('note_off', channel=1, note=53, velocity=95, time=40),
 Message('note_on', channel=1, note=64, velocity=95, time=56),
 Message('note_off', channel=1, note=62, velocity=95, time=24),
 Message('note_on', channel=1, note=65,

In [9]:
x.messages

[Message('note_on', channel=1, note=62, velocity=95, time=4608),
 Message('note_on', channel=1, note=64, velocity=95, time=96),
 Message('note_off', channel=1, note=62, velocity=95, time=54),
 Message('note_on', channel=1, note=65, velocity=95, time=138),
 Message('note_off', channel=1, note=64, velocity=95, time=32),
 Message('note_on', channel=1, note=64, velocity=95, time=1408),
 Message('note_off', channel=1, note=65, velocity=95, time=54),
 Message('note_on', channel=1, note=60, velocity=95, time=330),
 Message('note_off', channel=1, note=64, velocity=95, time=66),
 Message('note_on', channel=1, note=53, velocity=95, time=126),
 Message('note_off', channel=1, note=60, velocity=95, time=38),
 Message('note_on', channel=1, note=62, velocity=95, time=2266),
 Message('note_off', channel=1, note=53, velocity=95, time=40),
 Message('note_on', channel=1, note=64, velocity=95, time=56),
 Message('note_off', channel=1, note=62, velocity=95, time=24),
 Message('note_on', channel=1, note=65,

In [10]:
x.vector_encode()

[tensor([0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 1.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0

In [11]:
import torch.nn as nn

In [12]:
import torch

In [13]:
import torch.optim as optim

In [63]:
class MIDIModel(nn.Module):
    def __init__(self, hidden_layer_size=64):
        super().__init__()
        
        self.linear1 = nn.Linear(260, hidden_layer_size)
        self.sigmoid = nn.Sigmoid()
        self.linear2 = nn.Linear(hidden_layer_size, 130)
        self.logsoftmax = nn.LogSoftmax(dim=0)
    
    def forward(self, data):
        o = self.linear1(data)
        o = self.sigmoid(o)
        o = self.linear2(o)
        note_output = o[:,0:128]
        offset_output = o[:,128]
        duration_output = o[:,129]
        note_output = self.logsoftmax(note_output)
        return (note_output, offset_output, duration_output)

        

In [65]:
def generate_samples_per_song(song):
    vectors = song.vector_encode()
    samples = []
    for i in range(2, len(vectors)):
        samples.append((torch.cat((vectors[i-2], vectors[i-1])), (torch.LongTensor([song.sequence[i][0]]), vectors[i][-2], vectors[i][-1])))
        
    return samples

def generate_samples(songlist):
    samples = []
    for song in songlist:
        samples += generate_samples_per_song(song)
        
    return samples

In [66]:
class MIDINotesDataset(Dataset):
    def __init__(self, mididir, maximum=500):
        items = os.walk(mididir)
        
        self.filenames = []
        for (directory, _, files) in items:
            self.filenames += [os.path.join(directory, x) for x in files]
        
        mss = []
        count = 0
        for x in self.filenames:
            try:
                midifile = MidiFile(x)
                ms = MessageSequence(midifile)
            except:
                continue
                
            mss.append(ms)
            
            count += 1
            if count == maximum:
                break
            
        self.notes = generate_samples(mss)
        
    def __getitem__(self, i):
        return self.notes[i]
    
    def __len__(self):
        return len(self.notes)

In [128]:
dataset = MIDINotesDataset("lakh/lmd_full/3", maximum=400)

In [129]:
len(dataset)

102653

In [157]:
dataset[50000:50010]

[(tensor([0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000

In [131]:
dl = DataLoader(dataset, batch_size=25, shuffle=True)

In [132]:
dliter = iter(dl)

In [133]:
i, o = next(dliter)

In [134]:
i

tensor([[0.0000, 0.0000, 0.0000,  ..., 0.0000, 1.0000, 0.0250],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.1600, 0.0275],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0080],
        ...,
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.2560],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.4000, 0.0560],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.2800, 0.0070]])

In [135]:
i[:,0:128][3]

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.])

In [140]:
def train(data, epochs=10):
    mm = MIDIModel()
    optimizer = optim.SGD(mm.parameters(), lr=0.001, momentum=0.9)
    note_criterion = nn.NLLLoss()
    for epoch in range(epochs):
        loader = DataLoader(data, batch_size=25, shuffle=True)
        for i, o in loader:
            #print(i)
            #print(i.shape)
            #print(o)
            optimizer.zero_grad()
            (note_output, offset_output, duration_output) = mm(i)
            #print("no: {}, oo: {}, do: {}".format(note_output, offset_output, duration_output))
            #print(note_output, o[0].reshape((i.shape[0])))
            note_loss = torch.exp(-note_criterion(note_output, o[0].reshape((i.shape[0]))))
            offset_loss = torch.abs(o[1] - offset_output)
            duration_loss = torch.abs(o[2] - duration_output)
            #print("nl: {}, ol: {}, dl: {}".format(note_loss, offset_loss, duration_loss))
            loss = note_loss + offset_loss + duration_loss
            print(sum(loss))
            sum(loss).backward()
            optimizer.step()
    return mm

In [141]:
mm = MIDIModel()

In [142]:
samples = generate_samples([x])

In [143]:
samples

[(tensor([0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 1.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000

In [144]:
model = train(dataset)

tensor(16.7254, grad_fn=<AddBackward0>)
tensor(19.7973, grad_fn=<AddBackward0>)
tensor(15.2709, grad_fn=<AddBackward0>)
tensor(20.8045, grad_fn=<AddBackward0>)
tensor(21.1778, grad_fn=<AddBackward0>)
tensor(13.1482, grad_fn=<AddBackward0>)
tensor(15.8355, grad_fn=<AddBackward0>)
tensor(12.5586, grad_fn=<AddBackward0>)
tensor(16.9732, grad_fn=<AddBackward0>)
tensor(13.1595, grad_fn=<AddBackward0>)
tensor(13.1461, grad_fn=<AddBackward0>)
tensor(13.0499, grad_fn=<AddBackward0>)
tensor(18.5192, grad_fn=<AddBackward0>)
tensor(14.6281, grad_fn=<AddBackward0>)
tensor(23.1933, grad_fn=<AddBackward0>)
tensor(20.0394, grad_fn=<AddBackward0>)
tensor(13.5575, grad_fn=<AddBackward0>)
tensor(12.7449, grad_fn=<AddBackward0>)
tensor(19.8442, grad_fn=<AddBackward0>)
tensor(16.1468, grad_fn=<AddBackward0>)
tensor(14.7160, grad_fn=<AddBackward0>)
tensor(17.2734, grad_fn=<AddBackward0>)
tensor(14.2364, grad_fn=<AddBackward0>)
tensor(19.2939, grad_fn=<AddBackward0>)
tensor(16.3409, grad_fn=<AddBackward0>)


tensor(12.1735, grad_fn=<AddBackward0>)
tensor(10.5210, grad_fn=<AddBackward0>)
tensor(10.9111, grad_fn=<AddBackward0>)
tensor(13.7435, grad_fn=<AddBackward0>)
tensor(10.2626, grad_fn=<AddBackward0>)
tensor(13.9119, grad_fn=<AddBackward0>)
tensor(13.7994, grad_fn=<AddBackward0>)
tensor(21.6857, grad_fn=<AddBackward0>)
tensor(15.6849, grad_fn=<AddBackward0>)
tensor(13.5888, grad_fn=<AddBackward0>)
tensor(14.9683, grad_fn=<AddBackward0>)
tensor(16.2841, grad_fn=<AddBackward0>)
tensor(10.7211, grad_fn=<AddBackward0>)
tensor(9.1261, grad_fn=<AddBackward0>)
tensor(11.8931, grad_fn=<AddBackward0>)
tensor(16.1129, grad_fn=<AddBackward0>)
tensor(13.8998, grad_fn=<AddBackward0>)
tensor(15.1745, grad_fn=<AddBackward0>)
tensor(11.5024, grad_fn=<AddBackward0>)
tensor(18.5128, grad_fn=<AddBackward0>)
tensor(21.4127, grad_fn=<AddBackward0>)
tensor(6.9475, grad_fn=<AddBackward0>)
tensor(23.7147, grad_fn=<AddBackward0>)
tensor(26.0688, grad_fn=<AddBackward0>)
tensor(20.3716, grad_fn=<AddBackward0>)
te

tensor(10.6244, grad_fn=<AddBackward0>)
tensor(11.0002, grad_fn=<AddBackward0>)
tensor(12.6826, grad_fn=<AddBackward0>)
tensor(15.0525, grad_fn=<AddBackward0>)
tensor(17.6597, grad_fn=<AddBackward0>)
tensor(10.5532, grad_fn=<AddBackward0>)
tensor(16.3261, grad_fn=<AddBackward0>)
tensor(23.4259, grad_fn=<AddBackward0>)
tensor(14.5067, grad_fn=<AddBackward0>)
tensor(13.3519, grad_fn=<AddBackward0>)
tensor(14.7310, grad_fn=<AddBackward0>)
tensor(14.6933, grad_fn=<AddBackward0>)
tensor(13.4369, grad_fn=<AddBackward0>)
tensor(18.3362, grad_fn=<AddBackward0>)
tensor(10.0009, grad_fn=<AddBackward0>)
tensor(17.5394, grad_fn=<AddBackward0>)
tensor(24.7207, grad_fn=<AddBackward0>)
tensor(19.6493, grad_fn=<AddBackward0>)
tensor(10.7282, grad_fn=<AddBackward0>)
tensor(19.6903, grad_fn=<AddBackward0>)
tensor(21.2950, grad_fn=<AddBackward0>)
tensor(16.8103, grad_fn=<AddBackward0>)
tensor(9.3846, grad_fn=<AddBackward0>)
tensor(18.7929, grad_fn=<AddBackward0>)
tensor(18.8258, grad_fn=<AddBackward0>)
t

tensor(11.8872, grad_fn=<AddBackward0>)
tensor(12.5468, grad_fn=<AddBackward0>)
tensor(12.0794, grad_fn=<AddBackward0>)
tensor(10.9604, grad_fn=<AddBackward0>)
tensor(9.4810, grad_fn=<AddBackward0>)
tensor(10.4389, grad_fn=<AddBackward0>)
tensor(11.7309, grad_fn=<AddBackward0>)
tensor(12.2532, grad_fn=<AddBackward0>)
tensor(12.8474, grad_fn=<AddBackward0>)
tensor(10.0214, grad_fn=<AddBackward0>)
tensor(15.3956, grad_fn=<AddBackward0>)
tensor(14.7000, grad_fn=<AddBackward0>)
tensor(10.0313, grad_fn=<AddBackward0>)
tensor(11.4071, grad_fn=<AddBackward0>)
tensor(13.2306, grad_fn=<AddBackward0>)
tensor(12.4574, grad_fn=<AddBackward0>)
tensor(13.1013, grad_fn=<AddBackward0>)
tensor(10.9883, grad_fn=<AddBackward0>)
tensor(21.0229, grad_fn=<AddBackward0>)
tensor(14.3868, grad_fn=<AddBackward0>)
tensor(13.1521, grad_fn=<AddBackward0>)
tensor(16.0813, grad_fn=<AddBackward0>)
tensor(10.6575, grad_fn=<AddBackward0>)
tensor(14.9378, grad_fn=<AddBackward0>)
tensor(15.0871, grad_fn=<AddBackward0>)
t

tensor(10.4165, grad_fn=<AddBackward0>)
tensor(9.4229, grad_fn=<AddBackward0>)
tensor(9.0219, grad_fn=<AddBackward0>)
tensor(12.4514, grad_fn=<AddBackward0>)
tensor(14.7691, grad_fn=<AddBackward0>)
tensor(11.0263, grad_fn=<AddBackward0>)
tensor(10.3331, grad_fn=<AddBackward0>)
tensor(15.2147, grad_fn=<AddBackward0>)
tensor(9.8214, grad_fn=<AddBackward0>)
tensor(11.6649, grad_fn=<AddBackward0>)
tensor(13.4915, grad_fn=<AddBackward0>)
tensor(12.1267, grad_fn=<AddBackward0>)
tensor(12.8842, grad_fn=<AddBackward0>)
tensor(8.8688, grad_fn=<AddBackward0>)
tensor(13.6913, grad_fn=<AddBackward0>)
tensor(16.3401, grad_fn=<AddBackward0>)
tensor(10.0484, grad_fn=<AddBackward0>)
tensor(9.6042, grad_fn=<AddBackward0>)
tensor(12.8375, grad_fn=<AddBackward0>)
tensor(11.2122, grad_fn=<AddBackward0>)
tensor(12.7877, grad_fn=<AddBackward0>)
tensor(9.2129, grad_fn=<AddBackward0>)
tensor(9.1553, grad_fn=<AddBackward0>)
tensor(14.6403, grad_fn=<AddBackward0>)
tensor(10.4268, grad_fn=<AddBackward0>)
tensor(

...

tensor(7.6018, grad_fn=<AddBackward0>)
tensor(7.2722, grad_fn=<AddBackward0>)
tensor(5.5962, grad_fn=<AddBackward0>)
tensor(7.6679, grad_fn=<AddBackward0>)
tensor(7.2390, grad_fn=<AddBackward0>)
tensor(8.4877, grad_fn=<AddBackward0>)
tensor(7.1855, grad_fn=<AddBackward0>)
tensor(9.4159, grad_fn=<AddBackward0>)
tensor(8.0366, grad_fn=<AddBackward0>)
tensor(3.5716, grad_fn=<AddBackward0>)
tensor(7.1976, grad_fn=<AddBackward0>)
tensor(9.8370, grad_fn=<AddBackward0>)
tensor(6.5223, grad_fn=<AddBackward0>)
tensor(8.0375, grad_fn=<AddBackward0>)
tensor(5.3918, grad_fn=<AddBackward0>)
tensor(8.5889, grad_fn=<AddBackward0>)
tensor(6.2249, grad_fn=<AddBackward0>)
tensor(9.3011, grad_fn=<AddBackward0>)
tensor(6.9842, grad_fn=<AddBackward0>)
tensor(6.7906, grad_fn=<AddBackward0>)
tensor(6.8346, grad_fn=<AddBackward0>)
tensor(6.3466, grad_fn=<AddBackward0>)
tensor(4.6962, grad_fn=<AddBackward0>)
tensor(8.5527, grad_fn=<AddBackward0>)
tensor(5.8186, grad_fn=<AddBackward0>)
tensor(7.5884, grad_fn=<A

tensor(8.6215, grad_fn=<AddBackward0>)
tensor(5.6199, grad_fn=<AddBackward0>)
tensor(5.6298, grad_fn=<AddBackward0>)
tensor(11.6667, grad_fn=<AddBackward0>)
tensor(9.6072, grad_fn=<AddBackward0>)
tensor(6.7984, grad_fn=<AddBackward0>)
tensor(6.2641, grad_fn=<AddBackward0>)
tensor(6.6982, grad_fn=<AddBackward0>)
tensor(9.8758, grad_fn=<AddBackward0>)
tensor(7.5208, grad_fn=<AddBackward0>)
tensor(9.2952, grad_fn=<AddBackward0>)
tensor(7.5399, grad_fn=<AddBackward0>)
tensor(8.5882, grad_fn=<AddBackward0>)
tensor(9.2644, grad_fn=<AddBackward0>)
tensor(7.6076, grad_fn=<AddBackward0>)
tensor(7.0818, grad_fn=<AddBackward0>)
tensor(9.5036, grad_fn=<AddBackward0>)
tensor(6.3775, grad_fn=<AddBackward0>)
tensor(6.0051, grad_fn=<AddBackward0>)
tensor(6.7072, grad_fn=<AddBackward0>)
tensor(5.6287, grad_fn=<AddBackward0>)
tensor(8.6056, grad_fn=<AddBackward0>)
tensor(6.8350, grad_fn=<AddBackward0>)
tensor(10.8142, grad_fn=<AddBackward0>)
tensor(7.0310, grad_fn=<AddBackward0>)
tensor(7.9203, grad_fn=

tensor(7.9987, grad_fn=<AddBackward0>)
tensor(7.6537, grad_fn=<AddBackward0>)
tensor(12.6659, grad_fn=<AddBackward0>)
tensor(8.1673, grad_fn=<AddBackward0>)
tensor(7.7818, grad_fn=<AddBackward0>)
tensor(6.9992, grad_fn=<AddBackward0>)
tensor(7.5770, grad_fn=<AddBackward0>)
tensor(7.6755, grad_fn=<AddBackward0>)
tensor(7.6928, grad_fn=<AddBackward0>)
tensor(9.9070, grad_fn=<AddBackward0>)
tensor(5.9902, grad_fn=<AddBackward0>)
tensor(8.0727, grad_fn=<AddBackward0>)
tensor(7.0840, grad_fn=<AddBackward0>)
tensor(7.5899, grad_fn=<AddBackward0>)
tensor(8.7727, grad_fn=<AddBackward0>)
tensor(7.7839, grad_fn=<AddBackward0>)
tensor(6.9429, grad_fn=<AddBackward0>)
tensor(7.7911, grad_fn=<AddBackward0>)
tensor(5.0518, grad_fn=<AddBackward0>)
tensor(6.7765, grad_fn=<AddBackward0>)
tensor(7.2441, grad_fn=<AddBackward0>)
tensor(8.1225, grad_fn=<AddBackward0>)
tensor(9.7731, grad_fn=<AddBackward0>)
tensor(6.9881, grad_fn=<AddBackward0>)
tensor(7.2700, grad_fn=<AddBackward0>)
tensor(8.0302, grad_fn=<

tensor(7.8779, grad_fn=<AddBackward0>)
tensor(8.7271, grad_fn=<AddBackward0>)
tensor(8.5709, grad_fn=<AddBackward0>)
tensor(8.9100, grad_fn=<AddBackward0>)
tensor(6.5787, grad_fn=<AddBackward0>)
tensor(7.7856, grad_fn=<AddBackward0>)
tensor(9.0278, grad_fn=<AddBackward0>)
tensor(8.2654, grad_fn=<AddBackward0>)
tensor(7.1820, grad_fn=<AddBackward0>)
tensor(6.5581, grad_fn=<AddBackward0>)
tensor(4.1169, grad_fn=<AddBackward0>)
tensor(3.9982, grad_fn=<AddBackward0>)
tensor(6.4492, grad_fn=<AddBackward0>)
tensor(5.1559, grad_fn=<AddBackward0>)
tensor(9.2770, grad_fn=<AddBackward0>)
tensor(8.1346, grad_fn=<AddBackward0>)
tensor(8.1149, grad_fn=<AddBackward0>)
tensor(5.1494, grad_fn=<AddBackward0>)
tensor(8.0075, grad_fn=<AddBackward0>)
tensor(9.7063, grad_fn=<AddBackward0>)
tensor(8.7184, grad_fn=<AddBackward0>)
tensor(6.9235, grad_fn=<AddBackward0>)
tensor(8.0144, grad_fn=<AddBackward0>)
tensor(7.0011, grad_fn=<AddBackward0>)
tensor(7.8952, grad_fn=<AddBackward0>)
tensor(7.0036, grad_fn=<A

tensor(6.0578, grad_fn=<AddBackward0>)
tensor(6.8892, grad_fn=<AddBackward0>)
tensor(8.7026, grad_fn=<AddBackward0>)
tensor(7.5821, grad_fn=<AddBackward0>)
tensor(8.1760, grad_fn=<AddBackward0>)
tensor(5.8959, grad_fn=<AddBackward0>)
tensor(9.6700, grad_fn=<AddBackward0>)
tensor(5.7723, grad_fn=<AddBackward0>)
tensor(6.5179, grad_fn=<AddBackward0>)
tensor(7.6971, grad_fn=<AddBackward0>)
tensor(8.4356, grad_fn=<AddBackward0>)
tensor(6.9840, grad_fn=<AddBackward0>)
tensor(7.7544, grad_fn=<AddBackward0>)
tensor(7.8340, grad_fn=<AddBackward0>)
tensor(5.6620, grad_fn=<AddBackward0>)
tensor(9.1377, grad_fn=<AddBackward0>)
tensor(8.0788, grad_fn=<AddBackward0>)
tensor(8.1247, grad_fn=<AddBackward0>)
tensor(9.9784, grad_fn=<AddBackward0>)
tensor(7.4575, grad_fn=<AddBackward0>)
tensor(7.7640, grad_fn=<AddBackward0>)
tensor(6.5816, grad_fn=<AddBackward0>)
tensor(8.3482, grad_fn=<AddBackward0>)
tensor(6.3832, grad_fn=<AddBackward0>)
tensor(6.2619, grad_fn=<AddBackward0>)
tensor(7.9064, grad_fn=<A

In [145]:
samples[0][0]

tensor([0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 1.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 

In [158]:
# This is just to get the first two notes out of the development song.
vecs = x.vector_encode()

def generate_music(model, note1, note2, length=30):
    note_db = functional.one_hot(torch.arange(0, 128))
    newsong = [note1, note2]
    model.eval()
    with torch.no_grad():
        for i in range(length):
            (note_output, offset_output, duration_output) = model(torch.unsqueeze(torch.cat((note1, note2)), 0))
            print("note_output: {}".format(note_output))
            notenum = torch.argmax(note_output)
            note1 = note2
            print("testgen {} {} {}".format(note_db[notenum].clone().detach(), offset_output, duration_output))
            note2 = torch.cat((note_db[notenum].clone().detach(), torch.Tensor([offset_output]), 
                                                                               torch.Tensor([duration_output])))
            newsong.append(note2)
    return newsong

In [161]:
list(model.parameters()

[Parameter containing:
 tensor([[-0.0570, -0.0512, -0.0316,  ...,  0.0172, -1.1658, -0.6286],
         [-0.0381, -0.0414, -0.0535,  ...,  0.0236, -0.8628, -0.1151],
         [ 0.0572,  0.0138, -0.0594,  ...,  0.0266, -1.0409,  0.3956],
         ...,
         [ 0.0057, -0.0104, -0.0154,  ..., -0.0186, -0.8955, -0.1664],
         [ 0.0238, -0.0026,  0.0158,  ..., -0.0617, -0.9052, -0.5934],
         [-0.0457, -0.0435,  0.0375,  ...,  0.0418, -0.9924,  0.6775]],
        requires_grad=True),
 Parameter containing:
 tensor([-1.7760, -1.7889, -1.7440, -1.8302, -1.8731, -1.8384, -1.8381, -1.6711,
         -1.6265, -1.7430, -1.8282, -1.7229, -1.7278, -0.8057, -1.7270, -1.9049,
         -1.9539, -1.9765, -2.4672, -1.8622, -1.6826, -1.9680, -2.1216, -1.7808,
         -1.7062, -1.9121, -1.7857, -1.8301, -1.8386, -1.7216, -1.6739, -1.5887,
         -1.8684, -1.6181, -1.8398, -1.6536, -2.6793, -1.7131, -1.4995, -2.0745,
         -1.5766, -1.7599, -1.6479, -1.6203, -2.3629, -1.8055, -1.8228, -1.7028

In [159]:
newsong = generate_music(model, vecs[0], vecs[1])

note_output: tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.]])
testgen tensor([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 

In [151]:
newsong

[tensor([0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 1.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0

In [152]:
newsong[-1]

tensor([1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 

In [153]:
torch.argmax(newsong[-1][0:128])

tensor(0)

In [154]:
def reconvert_song(notetensors):
    return [(int(torch.argmax(x[0:128])), int(torch.floor(x[128] * 100)), int(torch.floor(x[129] * 4000))) for x in notetensors]

In [155]:
converted_song = reconvert_song(newsong)

In [156]:
converted_song

[(62, 54, 150),
 (64, 32, 224),
 (0, 25, 223),
 (0, 20, 166),
 (0, 28, 158),
 (0, 25, 151),
 (0, 28, 137),
 (0, 27, 130),
 (0, 28, 120),
 (0, 28, 113),
 (0, 28, 105),
 (0, 28, 99),
 (0, 28, 93),
 (0, 28, 87),
 (0, 28, 82),
 (0, 27, 78),
 (0, 27, 73),
 (0, 27, 69),
 (0, 27, 65),
 (0, 27, 62),
 (0, 27, 59),
 (0, 27, 55),
 (0, 27, 52),
 (0, 27, 50),
 (0, 27, 47),
 (0, 27, 45),
 (0, 27, 42),
 (0, 27, 40),
 (0, 27, 38),
 (0, 27, 36),
 (0, 27, 34),
 (0, 27, 33)]

In [49]:
x.sequence

[(62, 4608, 54, 150),
 (64, 4704, 32, 224),
 (65, 4896, 54, 1494),
 (64, 6336, 66, 450),
 (60, 6720, 38, 230),
 (53, 6912, 40, 2344),
 (62, 9216, 24, 120),
 (64, 9312, 24, 216),
 (65, 9504, 40, 1480),
 (64, 10944, 48, 432),
 (60, 11328, 40, 232),
 (69, 11520, 40, 1192),
 (67, 12672, 28, 1180),
 (62, 13824, 60, 156),
 (64, 13920, 40, 232),
 (65, 14112, 56, 1496),
 (64, 15552, 22, 406),
 (60, 15936, 30, 222),
 (55, 16128, 44, 2348),
 (53, 18432, 24, 2136),
 (50, 20544, 34, 226),
 (53, 20736, 38, 1190),
 (52, 21888, 1114, 1152),
 (52, 32256, 38, 38),
 (62, 32256, 54, 150),
 (64, 32352, 40, 232),
 (65, 32544, 64, 1504),
 (64, 33984, 66, 450),
 (60, 34368, 34, 226),
 (53, 34560, 32, 2336),
 (62, 36864, 48, 144),
 (64, 36960, 48, 240),
 (65, 37152, 60, 1500),
 (64, 38592, 48, 432),
 (60, 38976, 40, 232),
 (69, 39168, 44, 1196),
 (67, 40320, 24, 1176),
 (62, 41472, 80, 176),
 (64, 41568, 40, 232),
 (65, 41760, 56, 1496),
 (64, 43200, 34, 418),
 (60, 43584, 30, 222),
 (55, 43776, 60, 2364),
 (

In [51]:
def get_sequence_back(model_ouptut, starting_time):
    sequence = []
    for (note, offset, duration) in model_ouptut:
        sequence.append((note, starting_time, offset, duration))
        starting_time += duration - offset
        
    return sequence

note1: note_on .................. note_off
note2:             note_on ................... note_off
note3:                      note_on ... note_off
note4:                                                   note_on .......... note_off


In [52]:
recovered_sequence = get_sequence_back(converted_song, 4608)

In [53]:
recovered_sequence

[(62, 4608, 54, 150),
 (64, 4704, 32, 224),
 (50, 4896, 46, 311),
 (50, 5161, 44, 282),
 (50, 5399, 44, 274),
 (50, 5629, 44, 275),
 (50, 5860, 44, 275),
 (50, 6091, 44, 275),
 (50, 6322, 44, 275),
 (50, 6553, 44, 275),
 (50, 6784, 44, 275),
 (50, 7015, 44, 275),
 (50, 7246, 44, 275),
 (50, 7477, 44, 275),
 (50, 7708, 44, 275),
 (50, 7939, 44, 275),
 (50, 8170, 44, 275),
 (50, 8401, 44, 275),
 (50, 8632, 44, 275),
 (50, 8863, 44, 275),
 (50, 9094, 44, 275),
 (50, 9325, 44, 275),
 (50, 9556, 44, 275),
 (50, 9787, 44, 275),
 (50, 10018, 44, 275),
 (50, 10249, 44, 275),
 (50, 10480, 44, 275),
 (50, 10711, 44, 275),
 (50, 10942, 44, 275),
 (50, 11173, 44, 275),
 (50, 11404, 44, 275),
 (50, 11635, 44, 275)]

In [20]:
x = os.walk("lakh/lmd_full")

In [21]:
x

<generator object _walk at 0x7f673fb7b140>

In [22]:
next(x)

('lakh/lmd_full',
 ['f',
  'e',
  '0',
  '5',
  '8',
  '6',
  '7',
  '2',
  'd',
  'c',
  '9',
  'a',
  'b',
  '1',
  '4',
  '3'],
 [])

In [23]:
next(x)

('lakh/lmd_full/f',
 [],
 ['f8b9a90823dd6af25ac67eaf8fef6a43.mid',
  'f85a225a881d7e97643c2dc4e3a8fe9a.mid',
  'fdc9a30d6512b6165da48340c290a0c4.mid',
  'fee5e8ef07701433da3a79cfd9d82703.mid',
  'f2c6082099e6007dec7abbd60bcbf3b8.mid',
  'f8bf40a64acf9bfecadb76e01eda4660.mid',
  'f15fe73b0ee71771a82dca58773f4adf.mid',
  'f20cb8be7bfeca7ffc06900aa120f41f.mid',
  'fcae5290c66dc2f709f0466268b9d319.mid',
  'f58762009440865827b4d379ea113ae6.mid',
  'f287de78d1911307660d4dfdac248df7.mid',
  'fc3fae99db50064dbb3e8c890e475caf.mid',
  'fc28595b18e751d022cf75e61598a3a5.mid',
  'f70de8e5b2fd13047535d0d62423a805.mid',
  'f9ff6b69115d7a9862839208479be0b1.mid',
  'f40db875f53c31a94051f973509e448a.mid',
  'ff562d54a27b92eadae40b5adcee6466.mid',
  'fc9d5e737654406608dec635f19f1697.mid',
  'fe3b095f55c34a41cbba0c58c5dc3a2c.mid',
  'fcc9819dfe01ca8375f18e9252bca525.mid',
  'f1da2d486764c8340b20a10f1e228216.mid',
  'fe34ba18dc77b1a3b760e33495f42284.mid',
  'f378e169cc43a99767373cd14c8eeb8d.mid',
  'f20f1c

In [11]:
import glob

In [34]:
x = glob.glob("lakh/lmd_full//**//*.mid", recursive=True)

KeyboardInterrupt: 

In [ ]:
x